These solutions pertain to the CTF challenges available on <a href="https://pwnable.kr/play.php">pwnable.kr</a>!

<ol>
    <li> <a href="#toddlers-bottle">Toddler's Bottle</a> </li>
    <ul>
        <li><a href="#fd">fd</a> </li>
        <li><a href="#collision">collision</a> </li>
        <li><a href="#bof">bof</a> </li>
    </ul>
    <li> <a href="#rookiss">Rookiss</a> </li>
    <li> <a href="#grotesque">Grotesque</a> </li>
    <li> <a href="#hackerssecret">Hacker's Secret</a> </li>
</ol>

## <a name="toddlers-bottle"></a>Toddler's Bottle


#### <a name="fd"></a><font color="red">fd</font>
We first run the ls command to see what we are working with. We have three files:

* fd
* fd.c
* flag

As expected, the `flag` file does not have read permissions for us. So, let's examine the code. We can run the following command to output the contents of the `fd.c` file:

```shell
cat fd.c
```

Upon examining the contents of the program, it is clear that we need a way to fill the string "LETMEWIN\\n" into the `buf`. How? When we examine the read syscall closely, we see that it reads in from the file descriptor stored in `int fd`, which is essentially argv[1] converted to an int minus `0x1234`. Now, we need to understand the following building-blocks:

* STDIN is 0: If we manage to set `int fd` to 0, it will allow us to write directly from the terminal. This means whatever we type into the terminal will be entered into `buf`, which is great because we can type "LETMEWIN\\n".

However, we see that:

```C
int fd = atoi(argv[1]) - 0x1234;
```

To see what value `0x1234` (hex) is in decimal, we can simply type this into the shell:

```shell
printf "%d\n" 0x1234
```

Output: `4660`

So, to set `fd=0`, we have to pass in `4660` as the input. We run the program:

```shell
./fd 4660
```

and voila, we are allowed to enter a string. Now we can simply write "LETMEWIN\\n" (without the quotes, and \\n meaning an actual press of the return key), and behold the flag is:

`flag`: mommy! I think I know what a file descriptor is!!

<b>Question:</b> How are we not able to use `cat flag` to output the contents due to permission issues, but when we ourselves run this `fd` program with the correct input, we are able to access the contents of the file? After all, the process `fd` was run by us non-privileged users.

<b>Answer:</b> If you run `ls -l` and look at the permissions, you will notice an `s` in the owner's execute permissions. Something like: `-r-sr-x---`. This is called the `setuid` bit, and any program with the `setuid` bit set will run with the permissions of the owner. Therefore, when we correctly execute the program, it is able to print the content of the file "flag" as it is running with the permissions of the owner of "fd" file - who also happens to be the owner of the "flag" file. This happens instead of "fd" running with our permissions.

#### <a name="collision"></a><font color="red">collision</font>

In this challenge, we need to use the `col` program to output the contents of the `flag` file, despite not having 'read' permission. To achieve this, we need to understand how the program works, especially the `check_password` function. 

##### Analyzing the `col.c` Program
<b> High-Level Overview </b>
<br>1. Command-Line Argument: we pass a command-line argument, which acts as the 'password'.
<br>2. Password Length: the password must be exactly 20 bytes long.
<br>3. Password Check: the password is passed to the `check_password`, which verifies if it matches `0x21DD09EC`.

##### Understanding the Target Hash
The `check_password` function returns a `long` vlaue. To understand our target, let us convert the hexadecimal value `0x21DD09EC` to its decimal representation:
```sh
printf "%d\n" 0x21DD09EC
```

Output: `568134124`

<b> `check_password` function </b>
<br>The function takes a string input (a char pointer) and casts it into an `int` pointer. Here is how it processes the input:
<br>1. Pointer Evaluation: the function evaluates the pointer 4 bytes at a time, doing this 5 times (20 bytes total).
<br>2. Integer Addition: reads the first 4 bytes, interprets them as an `int`, adds this value to `res`.

<b>Code Explanation:</b>
<br>Here's the relevant code snippet:
```C
int* ip = (int*)p;
int i;
int res;
for(i = 0; i < 5; i++) {
    res += ip[i];
}
```

This code essentially finds different indices of the `int` pointer and adds the value that is pointed to at that memory address. An alternative approach could be:

```C
int* ip = (int*)p;
int i;
int res;
for(i = 0; i < 5; ip++) {
    res += *ip;
}
```
In this version, since `ip` is an `int` pointer, `ip++` will automatically increase the value by 4 bytes, checking 5 integer values.

##### Crafting the Solution
We need to pass values in the form of a string that, when evaluated as 5 `int`s, add up to `568134124`.
<b>Calculating Integer Values</b>
<br>First, let us divide `568134124` by 5:
$\frac{568134124}{5} = 113626824.8$

Since we cannot divide evenly, we adjust as follows:
<br>1. int 1: `113626824`
<br>2. Remaining Value: $568134124$ - $113626824$ = $454507300$
<br>3. Dividing Remaining Value: $\frac{454507300}{4} = 113626825$

Thus, our integers are:
<br> int 1: `113626824`
<br> int 2: `113626825`
<br> int 3: `113626825`
<br> int 4: `113626825`
<br> int 5: `113626825`

<b>Convertion to Hexadecimal</b>
<br>We have to convert these integes to hexadecimal:
<br> int 1: `113626824` = `0x6C5CEC8`
<br> int 2 - 5: `113626825` = `0x6C5CEC9`

To make this exactly 4 bytes, we add a leading zero:
<br> `0x06C5CEC8` and `0x06C5CEC9`

<b>Implementing the Solution</b> 
<br>We use perl to construct the input string:
```sh
./col $(perl -e 'print "\x06\xC5\xCE\xC9" x 4 . "\x06\xC5\xCE\xC8"')
```
This should convert to `int`s that add up to `568134124`. However, due to little-endian architecture, we need to reverse the byte order: 
```sh
./col $(perl -e 'print "\xC9\xCE\xC5\x06" x 4 . "\xC8\xCE\xC5\x06"')
```

##### Flag
`flag`: daddy! I just managed to create a hash collision :)

#### <a name="bof"></a><font color="red">bof</font>